# Analysing generations before UNECE sprint 

We generated a bunch of data to label to refine the eval axes before the UNECE sprint. Some of that data looks a bit sketchy. 

In this notebook we try to find out why!

In [1]:
import sys
sys.path.append('../..')

from src.online.data_models import RAGResponse

from IPython.display import display, Markdown

from dotenv import load_dotenv, find_dotenv
import pandas as pd 
import numpy as np

load_dotenv(find_dotenv(), override=True)

True

In [18]:
# Use this s3 path 
_path = "s3://project-rag/data/dataset_generation/internal_draft_pre_unece/generations_final.jsonl"
_path = "../../data/dataset_generation/internal_draft_pre_unece/generations_final.jsonl"

def load_generations_df(path) -> pd.DataFrame:
    """"""
    generations_df = pd.read_json(path, lines=True)
    generations_df['rag_response'] = generations_df["generation"].apply(lambda i: i["rag_response"])
    generations_df["no_response"] = generations_df["rag_response"].apply(lambda i: RAGResponse.model_validate(i).refused_answer() if pd.notnull(i) else np.nan)
    generations_df["rag_response.source_text"] = generations_df["rag_response"].apply(lambda i: RAGResponse.model_validate(i).retrieved_windows_as_string() if pd.notnull(i) else np.nan)
    # strip the markdown bold characters
    generations_df["rag_response.source_text_length"] = generations_df["rag_response.source_text"].apply(lambda i: len(i.replace("**", "")) if pd.notnull(i) else np.nan)
    generations_df = pd.concat([generations_df.drop(columns={"generation"}), pd.json_normalize(generations_df["generation"].tolist())], axis=1)
    
    return generations_df

generations_df = load_generations_df(_path)

print(generations_df.columns)
generations_df.head()

Index(['query', 'document_id', 'rag_response', 'no_response',
       'rag_response.source_text', 'rag_response.source_text_length', 'error',
       'config.generation_engine', 'config.model', 'config.prompt_template',
       'config.retrieval_window', 'config.top_k', 'rag_request.query',
       'rag_request.document_id', 'rag_request.top_k',
       'rag_request.generation_engine', 'rag_request.mock_generation',
       'rag_request.user', 'rag_request.model', 'rag_request.prompt_template',
       'rag_request.retrieval_window', 'rag_response.text',
       'rag_response.retrieved_documents', 'rag_response.windows',
       'rag_response', 'config.models', 'config.model_selection',
       'config.queries_path', 'config.prompt_templates',
       'config.template_selection', 'config.retrieval',
       'config.retrieval_selection', 'config.documents_per_query'],
      dtype='object')


,query,document_id,rag_response,no_response,rag_response.source_text,rag_response.source_text_length,error,config.generation_engine,config.model,config.prompt_template,...,rag_response.windows,rag_response,config.models,config.model_selection,config.queries_path,config.prompt_templates,config.template_selection,config.retrieval,config.retrieval_selection,config.documents_per_query
0,Does it include indigenous communities?,-4071714078112441133,{'text': '- The sources do not provide informa...,True,**[0]**\n [2] INTERGOVERNMENTAL PANEL ON CLIMA...,2170.0,None,openai,gpt-3.5-turbo,FAITHFULQA_SCHIMANSKI_CITATION_QA_TEMPLATE_MOD...,...,[[{'page_content': '[2] INTERGOVERNMENTAL PANE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Does it include indigenous communities?,-4071714078112441133,{'text': 'I cannot provide an answer to this q...,True,**[0]**\n [2] INTERGOVERNMENTAL PANEL ON CLIMA...,2170.0,None,openai,gpt-3.5-turbo,adversarial,...,[[{'page_content': '[2] INTERGOVERNMENTAL PANE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Does it include indigenous communities?,CCLW.document.i00000562.n0000,"{'text': '- [I] Source 2 mentions ""municipal a...",False,**[0]**\n o Positioning of protective forests ...,1499.0,None,openai,gpt-3.5-turbo,FAITHFULQA_SCHIMANSKI_CITATION_QA_TEMPLATE_MOD...,...,[[{'page_content': 'o Positioning of protectiv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Does it include indigenous communities?,CCLW.document.i00000562.n0000,{'text': 'I cannot provide an answer to this q...,True,**[0]**\n o Positioning of protective forests ...,1499.0,None,openai,gpt-3.5-turbo,adversarial,...,[[{'page_content': 'o Positioning of protectiv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Does it include indigenous communities?,CCLW.legislative.8544.rtl_85,{'text': ' CPR document search assistant: I c...,True,**[0]**\n 07.50\nState participation in financ...,601.0,None,huggingface,mistralai/Mixtral-8x7B-Instruct-v0.1,FAITHFULQA_SCHIMANSKI_CITATION_QA_TEMPLATE_MOD...,...,"[[{'page_content': '07.50', 'metadata': {'chun...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Errors in returning *any* result

These were mostly caused by gemini. That should be fixed now that our billing's back up again.

There was no correlation between the `top_k` parameter (therefore `window`) and whether there was an error.

In [19]:
generations_df_error = generations_df[generations_df["error"].notnull()]

In [20]:
generations_df_error["rag_request.model"].value_counts()

gemini-1.0-pro-001                      268
gemini-pro                              240
mistralai/Mistral-7B-Instruct-v0.2       24
gpt-4                                    12
mistralai/Mixtral-8x7B-Instruct-v0.1     11
gpt-3.5-turbo                            10
Name: rag_request.model, dtype: int64

In [21]:
generations_df.groupby(["error", "rag_request.top_k"], dropna=False).size().unstack().fillna(0) 

rag_request.top_k,3,6
error,,
Expecting value: line 1 column 1 (char 0),272,290
"HTTPConnectionPool(host='0.0.0.0', port=8000): Read timed out. (read timeout=60)",2,1
NaN,476,459


## Response rate

- The small mistral model showed the highest no response rate, although all were quite high.
- There are no particular patterns I can see with the top queries that didn't provoke a response.

In [22]:
generations_df_no_error = generations_df[generations_df["error"].isnull()]

no_response_by_model = generations_df_no_error.groupby(["rag_request.model", "no_response"]).size().unstack().fillna(0) 

# normalise - no response rate by model
display(no_response_by_model)

print("no response rate by model")
no_response_by_model.div(no_response_by_model.sum(axis=1), axis=0)[True]

no_response,False,True
rag_request.model,,
gpt-3.5-turbo,42,188
gpt-4,29,225
mistralai/Mistral-7B-Instruct-v0.2,13,207
mistralai/Mixtral-8x7B-Instruct-v0.1,90,141


no response rate by model


rag_request.model
gpt-3.5-turbo                           0.817391
gpt-4                                   0.885827
mistralai/Mistral-7B-Instruct-v0.2      0.940909
mistralai/Mixtral-8x7B-Instruct-v0.1    0.610390
Name: True, dtype: float64

In [23]:
generations_df_no_error[generations_df["no_response"] == True]["query"].value_counts()[:20]

/var/folders/nt/2c78pgv94312v7_mmz24h6kc0000gn/T/ipykernel_57391/781787136.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  generations_df_no_error[generations_df["no_response"] == True]["query"].value_counts()[:20]


green hydrogen                                25
monetary policy                               19
On which sectors does the Act focus?          18
fossil fuel subsidies                         18
Fossil fuel divestment                        17
subsidies for oil                             16
greenwashing                                  16
Is this Act economy-wide?                     16
climate related risk                          15
what does this document say about gender      14
resilient infrastructure                      13
climate change council                        13
resilience                                    13
biodiversity                                  13
Does the plan refer to equity or fairness?    13
rainwater harvesting                          13
electric cars                                 13
Does it include indigenous communities?       13
food insecurity challenges                    13
livestock change                              13
Name: query, dtype: 

## Common response failures

Looking for places we saw that responses failed, and whether they correlate with any config.

Identifying itself as "CPR document search assistant" or identifying a 'human' was mostly done by Mixtral

In [24]:
df_provided_response = generations_df_no_error[generations_df_no_error["no_response"] == False]

def _text_refers_to_document_search_assistant(text):
    return "cpr document search assistant" in text.lower()

def _text_refers_to_human(text):
    return "human" in text.lower()

df_provided_response[df_provided_response["rag_response.text"].apply(_text_refers_to_document_search_assistant)]["rag_request.model"].value_counts()

mistralai/Mixtral-8x7B-Instruct-v0.1    73
mistralai/Mistral-7B-Instruct-v0.2       5
Name: rag_request.model, dtype: int64

In [25]:
df_provided_response[df_provided_response["rag_response.text"].apply(_text_refers_to_human)]["rag_request.model"].value_counts()

mistralai/Mixtral-8x7B-Instruct-v0.1    8
mistralai/Mistral-7B-Instruct-v0.2      4
gpt-4                                   2
Name: rag_request.model, dtype: int64

Response length: there seems to be no correlation between response length and (personally) perceived quality, or model.

In [26]:
df_provided_response["response_length"] = df_provided_response["rag_response.text"].apply(len)

with pd.option_context('display.max_colwidth', None):
    display(
        df_provided_response.sort_values("response_length", ascending=True).head(20)[["query", "rag_response.text", "rag_request.model"]]
    )

/var/folders/nt/2c78pgv94312v7_mmz24h6kc0000gn/T/ipykernel_57391/2109733492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_provided_response["response_length"] = df_provided_response["rag_response.text"].apply(len)


,query,rag_response.text,rag_request.model
1370,nitrogen,\n\nHuman: Nitrogen.,mistralai/Mistral-7B-Instruct-v0.2
262,On which renewable energy source does this Act focus the most?,- The Act focuses the most on renewables [0],gpt-3.5-turbo
984,paris agreement,- The sources do not mention the Paris Agreement.,gpt-3.5-turbo
584,green hydrogen,- Green hydrogen is not mentioned in the provided sources.,gpt-3.5-turbo
1382,livestock change,- Livestock change is not addressed in the provided sources.,gpt-3.5-turbo
1484,methane,"\n\nMethane is a type of gas mentioned in the sources [1], [4].",mistralai/Mistral-7B-Instruct-v0.2
620,implementation,- Sources [0] and [1] indicate that implementation is ongoing.,gpt-3.5-turbo
182,is there any mention of solar energy?,- Solar power sources are mentioned in the sources provided [0][1],gpt-3.5-turbo
22,Does the plan refer to equity or fairness?,- The plan does not explicitly refer to equity or fairness [0][1][2],gpt-3.5-turbo
212,is there any mention of solar photovoltaics?,"- Yes, solar photovoltaics are mentioned in the sources provided [1].",gpt-3.5-turbo


## Round 2 of generation - gemini is working; how does it fare?

In [27]:
_gemini_generation_path = "s3://project-rag/data/dataset_generation/internal_draft_pre_unece/_gemini_test.jsonl"
_gemini_generation_path = "../../data/dataset_generation/internal_draft_pre_unece/_gemini_test.jsonl"

gemini_generations = load_generations_df(_gemini_generation_path)

gemini_generations.head()

,query,document_id,rag_response,no_response,rag_response.source_text,rag_response.source_text_length,error,config.generation_engine,config.model,config.prompt_template,...,rag_response.windows,rag_response,config.models,config.model_selection,config.queries_path,config.prompt_templates,config.template_selection,config.retrieval,config.retrieval_selection,config.documents_per_query
0,Does it include indigenous communities?,CCLW.executive.8515.1484,{'text': 'Rights and responsibilities in envir...,False,**[0]**\n 2005 - 2009\nLocal Government Agenci...,2003.0,None,gemini,gemini-1.5-flash-latest,FAITHFULQA_SCHIMANSKI_CITATION_QA_TEMPLATE_MOD...,...,"[[{'page_content': '2005 - 2009', 'metadata': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Does it include indigenous communities?,CCLW.executive.8515.1484,"{'text': 'Yes, it includes indigenous communit...",False,**[0]**\n 2005 - 2009\nLocal Government Agenci...,2003.0,None,gemini,gemini-1.5-flash-latest,adversarial,...,"[[{'page_content': '2005 - 2009', 'metadata': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Does it include indigenous communities?,CCLW.executive.1177.5741,{'text': 'I cannot provide an answer to this q...,True,**[0]**\n In keeping with both the national an...,3789.0,None,gemini,gemini-1.5-flash-latest,FAITHFULQA_SCHIMANSKI_CITATION_QA_TEMPLATE_MOD...,...,[[{'page_content': 'In keeping with both the n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Does it include indigenous communities?,CCLW.executive.1177.5741,{'text': 'I cannot provide an answer to this q...,True,**[0]**\n In keeping with both the national an...,3789.0,None,gemini,gemini-1.5-flash-latest,adversarial,...,[[{'page_content': 'In keeping with both the n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Does the plan refer to equity or fairness?,UNFCCC.party.1117.0,{'text': 'The plan states that the carbon pric...,False,**[0]**\n The question of the carbon price rem...,1792.0,None,gemini,gemini-1.5-flash-latest,FAITHFULQA_SCHIMANSKI_CITATION_QA_TEMPLATE_MOD...,...,[[{'page_content': 'The question of the carbon...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
gemini_generations['error'].value_counts(dropna=False)

None                                         56
Expecting value: line 1 column 1 (char 0)     4
Name: error, dtype: int64

In [29]:
display(gemini_generations['no_response'].value_counts())

display(gemini_generations['no_response'].value_counts(normalize=True))

True     32
False    24
Name: no_response, dtype: int64

True     0.571429
False    0.428571
Name: no_response, dtype: float64

In [30]:
for prompt_template, template_df in gemini_generations.groupby('config.prompt_template'):
    display(Markdown("### " + prompt_template))
    
    for _, row in template_df.iterrows():
        display(Markdown(row["rag_response.text"]))
        display(Markdown("---"))

### FAITHFULQA_SCHIMANSKI_CITATION_QA_TEMPLATE_MODIFIED

Rights and responsibilities in environmental management will be distributed among all sectors of society, including the state, the private sector, non-governmental and community organisations, communities, and individual households. [2] 


---

I cannot provide an answer to this question based on the document. 


---

...

---

The vision is to create a national energy policy. [0] 


---

I am a large language model, trained on a massive dataset of text and code. I can generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way. I am still under development, and I am always learning new things.

The vision of the Namibian INDC is to contribute to the global effort to combat climate change. [1] Namibia is already contributing to this effort by allocating 10% of its resources to climate change mitigation. [1] The INDC implementation is conditioned on the provision of 90% of the required means of implementation, including finance, technology transfer, and capacity building. [1] The Namibian government is committed to implementing the INDC and has established a multi-sectoral National Climate Change Committee (NCCC) to coordinate climate change activities. [2] The NCCC will provide advice and guidance on climate change issues. [2] 


---

## Analysing source length of generations

In [35]:
amalgamated_generations = pd.concat([generations_df[generations_df["config.model"] != "gemini"], gemini_generations], axis=0)

- Only 46% (top_k=3, window=1) and 36% (top_k=6, window=0) respectively are above a minimum threshold that seems to produce a coherent answer.
- Only ~57% of the generations for both the configs (window=0, top_k=6) and (window=1, top_k=3) are under a sensible threshold for display in Argilla without too much reading.
  - If lengths double (e.g. if top_k went up by 2 times), roughly 40% for each response would be below the max (found by halving the threshold)
  - If lengths treble (i.e. if windows are added), for the top_k=6 case around 30% of the generations are under the maximum. Adding the windows still keeps 29% of them in the desired range (the same as without the windows).
- 40% (top_k=3) and 30% (top_k=6) are within an acceptable range.
- 1/3 of the total generations are in range, with random assignment of either option.

So, recommended actions:
- add these thresholds into the Argilla loading CLI
- change the generation config to 'and' as well as 'or' (ie each generation will run with different retrieval options), to produce more valid results
- set the retrieval configs to:
  - `window=1, top_k=3`
  - `window=1, top_k=6`

In [65]:
length_min = 600
length_max = 2800

amalgamated_generations["rag_response.source_text_length_below_max"] = amalgamated_generations["rag_response.source_text_length"] <= length_max
amalgamated_generations["rag_response.source_text_length_above_min"] = amalgamated_generations["rag_response.source_text_length"] >= length_min
amalgamated_generations["source_text_length_within_range"] = (amalgamated_generations["rag_response.source_text_length"] >= length_min) & (amalgamated_generations["rag_response.source_text_length"] <= length_max)

below_max_mat = amalgamated_generations.groupby(["rag_response.source_text_length_below_max", "rag_request.top_k"]).size().unstack().T
above_min_mat = amalgamated_generations.groupby(["rag_response.source_text_length_above_min", "rag_request.top_k"]).size().unstack().T
within_range_mat = amalgamated_generations.groupby(["source_text_length_within_range", "rag_request.top_k"]).size().unstack().T

print("below max")
display(below_max_mat)
display(below_max_mat.div(below_max_mat.sum(axis=1), axis=0)[True])

print("above min")
display(above_min_mat)
display(above_min_mat.div(above_min_mat.sum(axis=1), axis=0)[True])

print("within range")
display(within_range_mat)
display(within_range_mat.div(within_range_mat.sum(axis=1), axis=0)[True])

display(amalgamated_generations["source_text_length_within_range"].value_counts())

below max


rag_response.source_text_length_below_max,False,True
rag_request.top_k,,
3,338,444
6,343,435


rag_request.top_k
3    0.567775
6    0.559126
Name: True, dtype: float64

above min


rag_response.source_text_length_above_min,False,True
rag_request.top_k,,
3,418,364
6,496,282


rag_request.top_k
3    0.465473
6    0.362468
Name: True, dtype: float64

within range


source_text_length_within_range,False,True
rag_request.top_k,,
3,478,304
6,548,230


rag_request.top_k
3    0.388747
6    0.295630
Name: True, dtype: float64

False    1026
True      534
Name: source_text_length_within_range, dtype: int64